# Spark Data Acquisition
### Kwame V. Taylor

This exercises uses the ```case.csv```, ```dept.csv```, and ```source.csv``` files from the San Antonio 311 call dataset.

### Exercises

1. Read the case, department, and source data into their own spark dataframes.


2. Let's see how writing to the local disk works in spark:
   * Write the code necessary to store the source data in both csv and json format, store these as ```sources_csv``` and ```sources_json```.
   * Inspect your folder structure. What do you notice?


3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

**Imports**

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

### Acquire data

In [2]:
source = spark.read.csv("source.csv", sep=",", header=True, inferSchema=True)
dept = spark.read.csv("dept.csv", sep=",", header=True, inferSchema=True)
case = spark.read.csv("case.csv", sep=",", header=True, inferSchema=True)

**Write data to disk**

In [3]:
source.write.csv("sources_csv", mode="overwrite")

In [4]:
source.write.json("sources_json", mode="overwrite")

I wrote this to the main (spark-exercises/) folder and got rid of the data/ folder. I prefer to have shorter file names over having my data in a folder.

```sources_csv``` is a folder containing these files:
* ```_SUCCESS```
* ```part-00000-10e448c2-f9f9-4610-8f8b-b98d2966fe59-c000.csv```

```sources_json``` is a folder containing these files:
* ```_SUCCESS```
* ```part-00000-2524418d-6636-414c-8962-06b2b3817975-c000.json```

**Inspect schemas and data types**

In [5]:
source.printSchema()
source.show(5)

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
|   106463| Richard Sanchez|
|   119403|  Betty De Hoyos|
|   119555|  Socorro Quiara|
+---------+----------------+
only showing top 5 rows



Both fields are strings. This is how I want the data, so I will leave it.

In [6]:
dept.printSchema()
dept.show(5)

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|
+--------------------+--------------------+----------------------+-------------------+
only showing top 5 r

```dept_subject_to_SLA``` needs to be converted into a boolean.

In [7]:
case.printSchema()
case.show(1, vertical=True)

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  


```case_opened_date```, ```case_closed_date```, and ```SLA_due_date``` need to be converted into Timestamps.

```case_late``` and ```case_closed``` need to be converted into booleans.

### Prepare data - ```dept```

In [8]:
# convert dept_subject_to_SLA to boolean

dept.groupby('dept_subject_to_SLA').count().show()

+-------------------+-----+
|dept_subject_to_SLA|count|
+-------------------+-----+
|                YES|   31|
|                 NO|    8|
+-------------------+-----+



In [9]:
dept = dept.withColumn('dept_subject_to_SLA', expr("dept_subject_to_SLA == 'YES'"))

In [10]:
dept.show(12)

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|               true|
|               Brush|Solid Waste Manag...|           Solid Waste|               true|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|               true|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|               true|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|               true|
|Code Enforcement ...|Code Enforcement ...|  DSD/Code Enforcement|               true|
|Code Enforcement ...|                null|  DSD/Code Enforcement|               true|
|   Dangerous Premise|Code Enforcement ...|  DSD/Code Enforcement|               true|
|Dangerous Premise...|Code Enforcement ...|

### Prepare data - ```case```

In [11]:
# rename SLA_due_date to case_due_date

case = case.withColumnRenamed("SLA_due_date", "case_due_date")

In [12]:
# convert case_late and case_closed to booleans

case.groupby('case_late', 'case_closed').count().show()

+---------+-----------+------+
|case_late|case_closed| count|
+---------+-----------+------+
|       NO|        YES|735616|
|      YES|        YES| 87978|
|       NO|         NO| 11585|
|      YES|         NO|  6525|
+---------+-----------+------+



In [13]:
case = case.withColumn("case_late", expr("case_late == 'YES'"))
case = case.withColumn("case_closed", expr("case_closed == 'YES'"))

In [14]:
case.printSchema()
case.show(2, vertical=True)

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- case_due_date: string (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 case_due_date        | 9/26/20 0:42         
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true               

**Convert the dates to timestamps**

In [15]:
fmt = "M/d/yy H:mm"

case.select(
    "case_opened_date",
    to_timestamp("case_opened_date", fmt)
).show(5)

+----------------+-----------------------------------------------+
|case_opened_date|to_timestamp(`case_opened_date`, 'M/d/yy H:mm')|
+----------------+-----------------------------------------------+
|     1/1/18 0:42|                            2018-01-01 00:42:00|
|     1/1/18 0:46|                            2018-01-01 00:46:00|
|     1/1/18 0:48|                            2018-01-01 00:48:00|
|     1/1/18 1:29|                            2018-01-01 01:29:00|
|     1/1/18 1:34|                            2018-01-01 01:34:00|
+----------------+-----------------------------------------------+
only showing top 5 rows



In [16]:
case = case.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
case = case.withColumn("case_closed_date", to_timestamp("case_closed_date", fmt))
case = case.withColumn("case_due_date", to_timestamp("case_due_date", fmt))

In [17]:
case.printSchema()
case.show(3, vertical=True)

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- case_due_date: timestamp (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true      

**Clean request_address**

In [18]:
case = case.withColumn("request_address", lower(trim("request_address")))

**Reformat council_district**

In [19]:
case = case.withColumn('council_district', format_string('%04d', col('council_district')))

**Add features**

In [20]:
# Add case_age, days_to_close, and case_lifetime features

case = case.withColumn("case_age", datediff(current_timestamp(), "case_opened_date"))
case = case.withColumn("days_to_close", datediff("case_closed_date", "case_opened_date"))
case = case.withColumn("case_lifetime", when(col("case_closed"), col("days_to_close")).otherwise(col("case_age")))

In [21]:
# Add zip code

case = case.withColumn("zipcode", regexp_extract("request_address", r"(\d+)$", 1))

In [22]:
case.show(3, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 0005                 
 case_age             | 1065                 
 days_to_close        | 0                    
 case_lifetime        | 0                    
 zipcode              | 78207                
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 2018-01-01

**Join dataframes**

In [23]:
case = case.join(dept, 'dept_division', 'left').join(source, 'source_id', 'left')

**Train, validate, test split**

In [24]:
train, test = case.randomSplit([.8, .2], seed=666)
train, validate, test = case.randomSplit([.7, .15, .15], seed=666)

In [25]:
train.show(5, vertical=True)

-RECORD 0--------------------------------------
 source_id              | 100137               
 dept_division          | 311 Call Center      
 case_id                | 1014263399           
 case_opened_date       | 2018-02-21 15:36:00  
 case_closed_date       | 2018-02-21 15:38:00  
 case_due_date          | 2018-03-02 15:36:00  
 case_late              | false                
 num_days_late          | -8.998854167000001   
 case_closed            | true                 
 service_request_type   | Compliment           
 SLA_days               | 9.0                  
 case_status            | Closed               
 request_address        | 927  donaldson av... 
 council_district       | 0007                 
 case_age               | 1014                 
 days_to_close          | 0                    
 case_lifetime          | 0                    
 zipcode                | 78228                
 dept_name              | Customer Service     
 standardized_dept_name | Customer Servi

In [26]:
print('train: ', train.count(), '| validate: ', validate.count(), '| test: ', test.count())

train:  598797 | validate:  128204 | test:  128268


### Exercises Continued

1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

2. How many Stray Animal cases are there?

3. How many service requests that are assigned to the Field Operations department (```dept_division```) are not classified as "Officer Standby" request type (```service_request_type```)?

4. Convert the ```council_district``` column to a string column.

5. Extract the year from the ```case_closed_date column```.

6. Convert ```num_days_late``` from days to hours in new columns ```num_hours_late```.

In [27]:
# How old is the latest (in terms of days past SLA) currently open issue?
# How long has the oldest (in terms of days since opened) currently opened issue been open?

case.createOrReplaceTempView('df')

spark.sql('''
SELECT DATEDIFF(current_timestamp, case_due_date) AS days_past_due
FROM df
WHERE NOT case_closed
ORDER BY days_past_due DESC
LIMIT 15
''').show()

+-------------+
|days_past_due|
+-------------+
|         1414|
|         1414|
|         1414|
|         1413|
|         1411|
|         1407|
|         1407|
|         1406|
|         1405|
|         1405|
|         1401|
|         1400|
|         1400|
|         1400|
|         1397|
+-------------+



In [28]:
# How many Stray Animal cases are there?
case.filter(case.service_request_type == 'Stray Animal').count()

27361

In [29]:
# How many service requests that are assigned to the Field Operations department (dept_division)
# are not classified as "Officer Standby" request type (service_request_type)?

case.filter(case.dept_division == 'Field Operations').filter(case.service_request_type != 'Officer Standby').count()

116295

In [30]:
# Convert the council_district column to a string column.

case.printSchema()

# It has already been converted so I'll leave it as is now.

root
 |-- source_id: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- case_due_date: timestamp (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = false)
 |-- case_age: integer (nullable = true)
 |-- days_to_close: integer (nullable = true)
 |-- case_lifetime: integer (nullable = true)
 |-- zipcode: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: boolean (nullable = true)
 |-- source_username: string (nullable

In [31]:
# Extract the year from the case_closed_date column.

year = case.select(year('case_closed_date'))
year.show()

+----------------------+
|year(case_closed_date)|
+----------------------+
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
+----------------------+
only showing top 20 rows



In [33]:
# Convert num_days_late from days to hours in new columns num_hours_late.

(
    case.withColumn('num_hours_late', case.num_days_late * 24)
    .select('num_days_late', 'num_hours_late')
    .show()
)

+-------------------+-------------------+
|      num_days_late|     num_hours_late|
+-------------------+-------------------+
| -998.5087616000001|     -23964.2102784|
|-2.0126041669999997|-48.302500007999996|
|       -3.022337963|      -72.536111112|
|       -15.01148148|      -360.27555552|
|0.37216435200000003|  8.931944448000001|
|       -29.74398148| -713.8555555199999|
|       -14.70673611|      -352.96166664|
|       -14.70662037|      -352.95888888|
|       -14.70662037|      -352.95888888|
|       -14.70649306|      -352.95583344|
|       -14.70649306|      -352.95583344|
|       -14.70636574|      -352.95277776|
|          -14.70625|-352.95000000000005|
|       -14.70636574|      -352.95277776|
|       -14.70623843|-352.94972232000003|
|-14.705891199999998|-352.94138879999997|
|       -14.70600694|      -352.94416656|
|       -14.70576389|      -352.93833336|
|       -14.70576389|      -352.93833336|
|       -14.70564815|       -352.9355556|
+-------------------+-------------

### Exercises Continued

7. Join the case data with the source and department data.

8. Are there any cases that do not have a request source?

9. What are the top 10 service request types in terms of number of requests?

10. What are the top 10 service request types in terms of average days late?

11. Does number of days late depend on department?

12. How do number of days late depend on department and request type?

In [34]:
# Join the case data with the source and department data.

case.show(1, vertical=True)

# It's already been joined.

-RECORD 0--------------------------------------
 source_id              | svcCRMLS             
 dept_division          | Field Operations     
 case_id                | 1014127332           
 case_opened_date       | 2018-01-01 00:42:00  
 case_closed_date       | 2018-01-01 12:29:00  
 case_due_date          | 2020-09-26 00:42:00  
 case_late              | false                
 num_days_late          | -998.5087616000001   
 case_closed            | true                 
 service_request_type   | Stray Animal         
 SLA_days               | 999.0                
 case_status            | Closed               
 request_address        | 2315  el paso st,... 
 council_district       | 0005                 
 case_age               | 1065                 
 days_to_close          | 0                    
 case_lifetime          | 0                    
 zipcode                | 78207                
 dept_name              | Animal Care Services 
 standardized_dept_name | Animal Care Se

In [35]:
# Are there any cases that do not have a request source?

case.filter(col('source_id').isNull()).show(vertical=True)

# This is the Spark df equivalent of .isnull().sum()

(0 rows)



In [36]:
# What are the top 10 service request types in terms of number of requests?

case.groupby('service_request_type').count().sort(col('count').desc()).show(10, truncate=False)

+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |89210|
|Overgrown Yard/Trash            |66403|
|Bandit Signs                    |32968|
|Damaged Cart                    |31163|
|Front Or Side Yard Parking      |28920|
|Stray Animal                    |27361|
|Aggressive Animal(Non-Critical) |25492|
|Cart Exchange Request           |22608|
|Junk Vehicle On Private Property|21649|
|Pot Hole Repair                 |20827|
+--------------------------------+-----+
only showing top 10 rows



In [37]:
# What are the top 10 service request types in terms of average days late?

case.where('case_late').groupBy('service_request_type').agg(mean('num_days_late').alias('n_days_late'), count('*').alias('n_cases')).sort(desc('n_days_late')).show(10, truncate=False)

+--------------------------------------+------------------+-------+
|service_request_type                  |n_days_late       |n_cases|
+--------------------------------------+------------------+-------+
|Zoning: Recycle Yard                  |210.89201994318182|132    |
|Zoning: Junk Yards                    |200.20517608494276|262    |
|Structure/Housing Maintenance         |190.20707698509807|51     |
|Donation Container Enforcement        |171.09115313942615|122    |
|Storage of Used Mattress              |163.96812829714287|7      |
|Labeling for Used Mattress            |162.43032902285717|7      |
|Record Keeping of Used Mattresses     |153.99724039428568|7      |
|Signage Requied for Sale of Used Mattr|151.63868055333333|12     |
|Traffic Signal Graffiti               |137.64583330000002|4      |
|License Requied Used Mattress Sales   |128.79828704142858|7      |
+--------------------------------------+------------------+-------+
only showing top 10 rows



In [38]:
# Does number of days late depend on department?

case.filter('case_late').groupby('dept_name').agg(mean('num_days_late').alias('days_late'), count('num_days_late').alias('n_cases_late')).sort('days_late').withColumn('days_late', round(col('days_late'), 1)).show(truncate=False)

+-------------------------+---------+------------+
|dept_name                |days_late|n_cases_late|
+-------------------------+---------+------------+
|Metro Health             |6.5      |854         |
|Solid Waste Management   |7.1      |33729       |
|Trans & Cap Improvements |10.7     |5529        |
|Parks and Recreation     |22.4     |3810        |
|Animal Care Services     |23.4     |23751       |
|Code Enforcement Services|48.1     |25467       |
|Development Services     |67.2     |840         |
|Customer Service         |88.2     |2035        |
|null                     |210.9    |132         |
+-------------------------+---------+------------+



In [39]:
# How do number of days late depend on department and request type?

case.filter("case_closed").groupby("standardized_dept_name", "service_request_type").agg(avg("num_days_late").alias("days_late"), count("*").alias("n_cases")).withColumn("days_late", round(col("days_late"), 1)).sort(asc("days_late")).show(25, truncate=False)

+------------------------+--------------------------------------+---------+-------+
|standardized_dept_name  |service_request_type                  |days_late|n_cases|
+------------------------+--------------------------------------+---------+-------+
|City Council            |Request for Research/Information      |null     |5      |
|Trans & Cap Improvements|Engineering Design                    |-1413.8  |1      |
|Trans & Cap Improvements|Signal Timing Modification By Engineer|-1352.0  |22     |
|Animal Care Services    |Stray Animal                          |-998.8   |27346  |
|Parks & Recreation      |Major Park Improvement Install        |-278.3   |271    |
|Trans & Cap Improvements|Sidewalk Cost Sharing Program         |-177.8   |131    |
|DSD/Code Enforcement    |Multi Tenant Exterior                 |-135.7   |84     |
|DSD/Code Enforcement    |CPS Energy Towers                     |-129.8   |511    |
|DSD/Code Enforcement    |CPS Energy Wood Poles                 |-129.3   |3